# Tutorial: Working with this stactools subpackage

Stactools ([docs](https://stactools.readthedocs.io/en/latest/), [source](https://github.com/stac-utils/stactools)) is a command line tool and library for working with [STAC](https://stacspec.org/), based on [PySTAC](https://github.com/stac-utils/pystac).

[Stactools dataset packages](https://github.com/stactools-packages) are add-ons for stactools that provide STAC functionality for specific datasets, such as [Sentinel 2](https://github.com/stactools-packages/sentinel2) and [Landsat](https://github.com/stactools-packages/landsat).

Stactools and its dataset packages can be accessed from the CLI or from within normal Python code. This notebook provides examples of both.

More information about the NALCMS dataset can be found [here](http://www.cec.org/north-american-land-change-monitoring-system/).

## 1. Using this notebook

The easiest way to use this notebook is to run it through `scripts/notebook`. This will create a Docker container Ready to run this notebook, which can be found in `docs/`.

If you wish to use this notebook outside of the container (such as on [mybinder.org](mybinder.org)) then please install the prerequisites using:

In [ ]:
!pip install stactools-nalcms

## 2. From the CLI

The first thing we can do is check that the `stac` and `stac-nalcms` CLI tools are installed and explore the options. Notice the inclusion of the `nalcms` command for `stac`:

In [1]:
!stac

Usage: stac [OPTIONS] COMMAND [ARGS]...

Options:
  -v, --verbose  Use verbose mode
  -q, --quiet    Use quiet mode (no output)
  --help         Show this message and exit.

Commands:
  copy         Copy a STAC Catalog
  describe     Prints out a list of all catalogs, collections and items in
               this STAC.

  info         Display info about a static STAC catalog.
  layout       Reformat the layout of a STAC based on templating.
  merge        Merge items from one STAC into another.
  move-assets  Move or copy assets in a STAC to the Item locations.
  nalcms       Commands for working with NALCMS data.
  validate     Validate a stac object.
  version      Display version info.


You can now explore the STAC dataset package commands to ingest and describe the data

In [2]:
!stac nalcms --help

Usage: stac nalcms [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  create-cog         Transform Geotiff to Cloud-Optimized Geotiff.
  create-collection  Creates STAC collections for NALCMS data.
  create-item        Create a STAC item for a given region, GSD and year.


And more specific help with an individual command.

In [3]:
!stac nalcms create-collection --help

Usage: stac nalcms create-collection [OPTIONS]

  Creates a STAC Collection for each mapped dataset from the North American
  Land Classification Monitoring System. Args:     destination (str):
  Directory used to store the STAC collections. Returns:     Callable

Options:
  -d, --destination TEXT  The output directory for the STAC Collection json.
                          [required]

  --help                  Show this message and exit.


In [4]:
!stac nalcms create-item --help

Usage: stac nalcms create-item [OPTIONS]

  Creates a STAC Item

  Args:     destination (str): The output directory for the STAC json.
  source (str): The input COG to create the item from.     region (str): The
  region covered by the STAC Item.     gsd (int, float): The ground sampling
  distance of the STAC Item.     year (str): The year or range of years
  covered by the STAC Item.

Options:
  -d, --destination TEXT          The output directory for the STAC json.
                                  [required]

  -s, --source TEXT               The input COG to create the item from.
  -r, --region [CAN|USA|MEX|NA|ASK|HI]
                                  The region covered by the STAC Item.
  -g, --gsd [30|250]
  -y, --year [2005-2010|2005|2010-2015|2015|2010]
                                  The year or range of years covered by the
                                  STAC Item.

  --help                          Show this message and exit.


The metadata necessary to create the root STAC collection, regional subcollections and all items is contained within the dataset package, so only an output directory is required:

In [9]:
!stac nalcms create-collection -d "."
!ls
!head ./collection.json
!head ./NALCMS_yearly/collection.json
!head ./NALCMS_yearly/USA_2010_30m/USA_2010_30m.json

CAN_2010-2015_30m.json	index.rst			    Makefile
collection.json		installation_and_basic_usage.ipynb  NALCMS_change
conf.py			make.bat			    NALCMS_yearly
{
  "type": "Collection",
  "id": "nalcms",
  "stac_version": "1.0.0",
  "description": "NALCMS products can be used for a variety of applications, including: carbon sequestration analysis, wildlife habitat mapping, ecosystem monitoring, environmental planning, water quality assessments, and evaluation of biofuels production potential.The maps produced under this initiative represent land cover in 2005, 2010, and 2015, & are based on either Moderate Resolution Imaging Spectroradiometer (MODIS) satellite imagery monthly composites at 250 m spatial resolution; Landsat-7; or RapidEye satellite imagery at 30 m spatial resolution. The NALCMS\u2019 nineteen land cover classes are based on the Land Cover Classification System (LCCS) standard developed by the Food & Agriculture Organization (FAO) of the United Nations.",
  "links": [
    {
  

Items exist for most combinations of: 
- Region (CAN|USA|MEX|NA|ASK|HI)
- Ground sampling distance (GSD) (30|250)
- Year or land classification change between years (2005-2010|2005|2010-2015|2015|2010)

Which combinations exist can be found [here](http://www.cec.org/north-american-land-change-monitoring-system/) or by looking at STAC structure created with `create-collection`.

`create-item` will create the item for (NA, 30, 2010-2015) by default. Below is an example of creating the item for (CAN, 30, 2010-2015). The path to an associated COG can be optionally included as a data asset using the `--source` option.

In [6]:
!stac nalcms create-item -d "." -r CAN -g 30 -y 2010-2015
!head ./CAN_2010-2015_30m.json

{
  "type": "Feature",
  "stac_version": "1.0.0",
  "id": "CAN_2010-2015_30m",
  "properties": {
    "title": "CAN land cover change (2010-2015, 30 m)",
    "description": "Land cover change for 2010-2015 over Canada (30 m)",
    "start_datetime": "2010-01-01T00:00:00Z",
    "end_datetime": "2015-12-31T00:00:00Z",
    "gsd": 30.0,


## 3. As a Python module

So far we've used IPython [line magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) to work with stactools packages on the command line from this notebook, but it's also possible to use them within Python scripts.

In [13]:
from stactools.nalcms import stac
from stactools.nalcms.constants import GSDS, REGIONS, YEARS, PERIODS
import os
import itertools as it

Like above it's possible to create the complete STAC from the metadata within the dataset package:

In [14]:
root_col = stac.create_nalcms_collection()

for per, years in PERIODS.items():
    combos = it.product(REGIONS.keys(), GSDS, years)
    period = stac.create_period_collection(per)
    root_col.add_child(period)

    for reg, gsd, year in combos:
        item = stac.create_item(reg, gsd, year, "")
        if item is not None:
            period.add_item(item)

In [15]:
root_col.to_dict()

{'type': 'Collection',
 'id': 'nalcms',
 'stac_version': '1.0.0',
 'description': 'NALCMS products can be used for a variety of applications, including: carbon sequestration analysis, wildlife habitat mapping, ecosystem monitoring, environmental planning, water quality assessments, and evaluation of biofuels production potential.The maps produced under this initiative represent land cover in 2005, 2010, and 2015, & are based on either Moderate Resolution Imaging Spectroradiometer (MODIS) satellite imagery monthly composites at 250 m spatial resolution; Landsat-7; or RapidEye satellite imagery at 30 m spatial resolution. The NALCMS’ nineteen land cover classes are based on the Land Cover Classification System (LCCS) standard developed by the Food & Agriculture Organization (FAO) of the United Nations.',
 'links': [{'rel': <RelType.ROOT: 'root'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <ScientificRelType.CITE_AS: 'cite-as'>,
   'href': 'https://doi.or

In [16]:
next(root_col.get_all_items()).to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0',
 'id': 'CAN_2010_30m',
 'properties': {'title': 'CAN land cover (2010, 30 m)',
  'description': 'Land cover for 2010 over Canada (30 m)',
  'start_datetime': '2010-01-01T00:00:00Z',
  'end_datetime': '2010-12-31T00:00:00Z',
  'gsd': 30.0,
  'proj:epsg': None,
  'proj:transform': [30.0,
   0.0,
   -2315310.0000000037,
   0.0,
   -30.0,
   4309999.999999997,
   0.0,
   0.0,
   1.0],
  'proj:bbox': [-2315310.0000000037,
   -250300.0000000028,
   3345089.9999999963,
   4309999.999999997],
  'proj:wkt2': 'PROJCS["WGS_1984_Lambert_Azimuthal_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",45],PARAMETER["longitude_of_center",-100],PARAMETER["false_easting",0],PARAMETER["false_northin

And similarly, it's possible to create an individual item for a given region/GSD/year(s):

In [17]:
item = stac.create_item("CAN", "30", "2010", source="path/to/cog.tif")
item.to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0',
 'id': 'CAN_2010_30m',
 'properties': {'title': 'CAN land cover (2010, 30 m)',
  'description': 'Land cover for 2010 over Canada (30 m)',
  'start_datetime': '2010-01-01T00:00:00Z',
  'end_datetime': '2010-12-31T00:00:00Z',
  'gsd': 30.0,
  'proj:epsg': None,
  'proj:transform': [30.0,
   0.0,
   -2315310.0000000037,
   0.0,
   -30.0,
   4309999.999999997,
   0.0,
   0.0,
   1.0],
  'proj:bbox': [-2315310.0000000037,
   -250300.0000000028,
   3345089.9999999963,
   4309999.999999997],
  'proj:wkt2': 'PROJCS["WGS_1984_Lambert_Azimuthal_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",45],PARAMETER["longitude_of_center",-100],PARAMETER["false_easting",0],PARAMETER["false_northin